## Notebook used to create and test the models 

In [15]:
import pandas as pd
import seaborn as sns
import matplotlib
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report,confusion_matrix
from sklearn.metrics import mean_absolute_error,mean_squared_error
from ast import literal_eval
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Activation,Dropout
from tensorflow.keras.layers import LSTM,Embedding
import numpy as np
import math

In [3]:
POLLUTION = pd.read_csv("data/TextPollution.csv")
POLLUTION.head()

,Unnamed: 0,headlines,variations
0,0,"[0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, ...","[1.183673469387755, 1.0337662337662337, 1.1432..."
1,1,"[0.0, 1.0, 0.0, 0.0, 0.0, 4.0, 0.0, 0.0, 0.0, ...","[0.5344827586206896, 0.8710217755443886, 0.547..."
2,2,"[0.0, 1.0, 5.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[1.1935483870967742, 0.8052884615384617, 1.155..."
3,3,"[0.0, 6.0, 3.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, ...","[0.8648648648648649, 0.9253731343283581, 1.144..."
4,4,"[0.0, 2.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, ...","[1.75, 1.6806451612903226, 1.5001070893124868,..."


### gets both the features and labels matrices from the dataset and scales the features matrix

In [4]:
def get_matrices():
    
    headlines =  POLLUTION['headlines'].apply(literal_eval)
    
    matrix_headlines = list()
    
    for i in headlines.keys():
        matrix_headlines.append([float(x) for x in headlines[i]])
    
    variations = POLLUTION['variations'].apply(literal_eval)

    matrix_variations = list()
    
    for i in variations.keys():
        matrix_variations.append([float(x) for x in variations[i]])
    
    return  (matrix_headlines,matrix_variations)

def scale_matrix(matrix):
    
    standard_scaler = preprocessing.StandardScaler()
    standard_scaler.fit(matrix)
    matrix = standard_scaler.transform(matrix)
    
    return [list(x) for x in matrix]

headlines,variations = get_matrices()
headlines = scale_matrix(headlines)
POLLUTION['headlines'] = headlines
POLLUTION['variations'] = variations
##print(headlines)

In [5]:
POLLUTION.head()

,Unnamed: 0,headlines,variations
0,0,"[-0.7538061585358591, -0.9961555380372183, -0....","[1.183673469387755, 1.0337662337662337, 1.1432..."
1,1,"[-0.7538061585358591, -0.34015067152490386, -0...","[0.5344827586206896, 0.8710217755443886, 0.547..."
2,2,"[-0.7538061585358591, -0.34015067152490386, 1....","[1.1935483870967742, 0.8052884615384617, 1.155..."
3,3,"[-0.7538061585358591, 2.9398736610366685, 0.67...","[0.8648648648648649, 0.9253731343283581, 1.144..."
4,4,"[-0.7538061585358591, 0.31585419498741063, -0....","[1.75, 1.6806451612903226, 1.5001070893124868,..."


In [6]:
X = np.array(POLLUTION['headlines'].to_list())
y = np.array(POLLUTION['variations'].to_list())

In [7]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.3)
print("elements in training set: "  +  str(len(X_train)))
print("elements in testing  set: " +  str(len(X_test)))

elements in training set: 18
elements in testing  set: 9


In [43]:
def run_tests(y_test,predictions):
    mae = mean_absolute_error(y_test,predictions)
    mse = mean_squared_error(y_test,predictions)
    rmse = math.sqrt(mean_squared_error(y_test,predictions))
    
    print("MEAN ABSOLUTE ERROR: " + str(mae))
    print("MEAN SQUARED ERROR: " + str(mse))
    print("ROOT MEAN SQUARED ERROR: " + str(rmse))
    
    return (mae,mse,rmse)

### Creates the dnn model for a given input and output size

In [32]:
def create_model_dnn(input_size,output_size):
    model = Sequential()
    model.add(Dense(64,input_size))
    model.add(Activation('relu'))
    model.add(Dropout(0.5))
    model.add(Dense(64))
    model.add(Activation('relu'))
    model.add(Dropout(0.5))
    model.add(Dense(output_size,activation='softmax'))
    
    model.compile(optimizer='rmsprop',loss='mse',metrics=['mean_squared_error'])
    
    return model

### Creates a lstm model for a given input and output size 
### Working can be improved

In [36]:
def create_model_lstm(input_size,output_size):
    model = Sequential()
    model.add(Embedding(input_size,output_size))
    model.add(LSTM(128))
    model.add(Dropout(0.5))
    model.add(Dense(output_size,activation='sigmoid'))
    
    model.compile(loss='mse',optimizer='rmsprop',
                  metrics=['mean_squared_error'])
    
    return model

In [17]:
dnn = create_model_dnn(len(X_train[0]),len(y_train[0]))

In [18]:
dnn.fit(X_train,y_train,batch_size=32,epochs=100)

Epoch 1/100
18/18 [==============================] - 0s 8ms/sample - loss: 2.7462 - mean_squared_error: 2.7462
Epoch 2/100
18/18 [==============================] - 0s 325us/sample - loss: 2.7084 - mean_squared_error: 2.7084
Epoch 3/100
18/18 [==============================] - 0s 325us/sample - loss: 2.7205 - mean_squared_error: 2.7205
Epoch 4/100
18/18 [==============================] - 0s 271us/sample - loss: 2.6639 - mean_squared_error: 2.6639
Epoch 5/100
18/18 [==============================] - 0s 271us/sample - loss: 2.7182 - mean_squared_error: 2.7182
Epoch 6/100
18/18 [==============================] - 0s 326us/sample - loss: 2.6662 - mean_squared_error: 2.6662
Epoch 7/100
18/18 [==============================] - 0s 325us/sample - loss: 2.6856 - mean_squared_error: 2.6856
Epoch 8/100
18/18 [==============================] - 0s 542us/sample - loss: 2.6200 - mean_squared_error: 2.6200
Epoch 9/100
18/18 [==============================] - 0s 271us/sample - loss: 2.5801 - mean_squared

Epoch 73/100
18/18 [==============================] - 0s 380us/sample - loss: 2.4449 - mean_squared_error: 2.4449
Epoch 74/100
18/18 [==============================] - 0s 271us/sample - loss: 2.4569 - mean_squared_error: 2.4569
Epoch 75/100
18/18 [==============================] - 0s 434us/sample - loss: 2.4319 - mean_squared_error: 2.4319
Epoch 76/100
18/18 [==============================] - 0s 271us/sample - loss: 2.4680 - mean_squared_error: 2.4680
Epoch 77/100
18/18 [==============================] - 0s 379us/sample - loss: 2.4391 - mean_squared_error: 2.4391
Epoch 78/100
18/18 [==============================] - 0s 434us/sample - loss: 2.4719 - mean_squared_error: 2.4719
Epoch 79/100
18/18 [==============================] - 0s 271us/sample - loss: 2.4551 - mean_squared_error: 2.4551
Epoch 80/100
18/18 [==============================] - 0s 325us/sample - loss: 2.4420 - mean_squared_error: 2.4420
Epoch 81/100
18/18 [==============================] - 0s 325us/sample - loss: 2.4310 - m

In [19]:
predictions = dnn.predict(X_test)

In [28]:
run_tests(y_test,predictions)

MEAN ABSOLUTE ERROR: 0.8346067586141983
MEAN SQUARED ERROR: 0.9923563180945505
ROO MEAN SQUARED ERROR0.9961708277672814


(0.8346067586141983, 0.9923563180945505, 0.9961708277672814)

In [38]:
lstm = create_model_lstm(len(X_train[0]),len(y_train[0]))

In [46]:
lstm.fit(X_train,y_train,batch_size=16,epochs=100)

Epoch 1/100
18/18 [==============================] - 2s 91ms/sample - loss: 1.6444 - mean_squared_error: 1.6444
Epoch 2/100
18/18 [==============================] - 2s 89ms/sample - loss: 1.6448 - mean_squared_error: 1.6448
Epoch 3/100
18/18 [==============================] - 1s 71ms/sample - loss: 1.6443 - mean_squared_error: 1.6443
Epoch 4/100
18/18 [==============================] - 1s 75ms/sample - loss: 1.6444 - mean_squared_error: 1.6444
Epoch 5/100
18/18 [==============================] - 1s 70ms/sample - loss: 1.6444 - mean_squared_error: 1.6444
Epoch 6/100
18/18 [==============================] - 1s 75ms/sample - loss: 1.6444 - mean_squared_error: 1.6444
Epoch 7/100
18/18 [==============================] - 1s 77ms/sample - loss: 1.6443 - mean_squared_error: 1.6443
Epoch 8/100
18/18 [==============================] - 1s 77ms/sample - loss: 1.6442 - mean_squared_error: 1.6442
Epoch 9/100
18/18 [==============================] - 1s 64ms/sample - loss: 1.6443 - mean_squared_error:

18/18 [==============================] - 1s 60ms/sample - loss: 1.6442 - mean_squared_error: 1.6442
Epoch 74/100
18/18 [==============================] - 1s 53ms/sample - loss: 1.6442 - mean_squared_error: 1.6442
Epoch 75/100
18/18 [==============================] - 1s 61ms/sample - loss: 1.6442 - mean_squared_error: 1.6442
Epoch 76/100
18/18 [==============================] - 1s 68ms/sample - loss: 1.6442 - mean_squared_error: 1.6442
Epoch 77/100
18/18 [==============================] - 1s 64ms/sample - loss: 1.6442 - mean_squared_error: 1.6442
Epoch 78/100
18/18 [==============================] - 1s 60ms/sample - loss: 1.6442 - mean_squared_error: 1.6442
Epoch 79/100
18/18 [==============================] - 1s 74ms/sample - loss: 1.6442 - mean_squared_error: 1.6442
Epoch 80/100
18/18 [==============================] - 1s 61ms/sample - loss: 1.6442 - mean_squared_error: 1.6442
Epoch 81/100
18/18 [==============================] - 1s 61ms/sample - loss: 1.6442 - mean_squared_error: 1.6

In [47]:
predictions = lstm.predict(X_test)

In [48]:
run_tests(y_test,predictions)

MEAN ABSOLUTE ERROR: 0.33790203579180805
MEAN SQUARED ERROR: 0.2687145946571488
ROO MEAN SQUARED ERROR: 0.5183768847635365


(0.33790203579180805, 0.2687145946571488, 0.5183768847635365)